<a href="https://colab.research.google.com/github/starirene9/DeepLearningAssignment/blob/main/%5B%EB%94%A5%EB%9F%AC%EB%8B%9D_%EC%B5%9C%EC%8B%A0_%EA%B8%B0%EC%88%A0%5D%5B2%EC%A1%B0%5D%5B11%EC%A3%BC%EC%B0%A8%5D%EC%9C%A0%ED%8A%9C%EB%B8%8C_%EC%9A%94%EC%95%BD_%EC%95%B1_%EA%B3%A0%EB%8F%84%ED%99%94_%2B_%ED%94%84%EB%A0%88%EC%9E%84_%EC%BA%A1%EC%B2%98_%EB%B0%8F_%EC%84%A4%EB%AA%85.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1: 설치 및 임포트
!pip install git+https://github.com/openai/whisper.git pytubefix pydub moviepy gradio google-genai pysrt ffmpeg-python tiktoken blobfile

!apt-get update
!apt-get install -y imagemagick

!sudo sed -i '/<policy domain="path" rights="none" pattern="@\*"/d' /etc/ImageMagick-6/policy.xml
# ImageMagick 정책 파일 수정 (moviepy 사용 시 충돌 방지)

# 나눔글꼴 설치 (한글 자막이 moviepy에서 깨지는 것 방지용)
!apt-get install -qq fonts-nanum &> /dev/null
!fc-cache -fv &> /dev/null    # 폰트 캐시 갱신

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-ky5ljuxr
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-ky5ljuxr
  Resolved https://github.com/openai/whisper.git to commit dd985ac4b90cafeef8712f2998d62c59c3e62d22
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.4/104.4 kB 2.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.5/731.5 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 MB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 120.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
# 1: 설치 및 임포트
import os
import re
import time
from tqdm import tqdm
import random
import tempfile
import pysrt
from pytubefix import YouTube, request
from pydub import AudioSegment
import whisper
from moviepy.editor import VideoFileClip, TextClip, CompositeVideoClip, ColorClip
import ffmpeg
import gradio as gr
from google import genai
import concurrent.futures
from functools import partial
import cv2
import shutil
from PIL import Image
from glob import glob
from transformers import AutoModelForCausalLM, AutoProcessor

  if event.key is 'enter':



In [ ]:
# 2: 모델 로드 및 임시 디렉토리 생성
request.default_client = "WEB"
transcriber = whisper.load_model("small")
LANG_CODE = {"한국어":"ko", "English":"en", "日本語":"ja", "Español": "es"}

# 설정 디렉토리
VIDEO_DIR = "./videos"
SUB_VIDEO_DIR = "./subtitles_videos"
CAPTURE_DIR = "./captures"
SUB_DIR = "./subtitles"
AUDIO_DIR = "./audio"
os.makedirs(VIDEO_DIR, exist_ok=True)
os.makedirs(CAPTURE_DIR, exist_ok=True)
os.makedirs(SUB_VIDEO_DIR, exist_ok=True)
os.makedirs(SUB_DIR, exist_ok=True)
os.makedirs(AUDIO_DIR, exist_ok=True)

import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

ImageToText_model_path = "moonshotai/Kimi-VL-A3B-Thinking"
ImageToText = AutoModelForCausalLM.from_pretrained(
    ImageToText_model_path, device_map={"": device}, torch_dtype="auto", trust_remote_code=True
)
# ImageToText = AutoModelForCausalLM.from_pretrained(ImageToText_model_path, torch_dtype="auto", device_map="auto", trust_remote_code=True,)
ImageToTextProcessor = AutoProcessor.from_pretrained(ImageToText_model_path, trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
# base video 첨부
!gdown https://drive.google.com/uc?id=1ggZc7kJMcHRiVH0EUAmQAOJYWgfLi1uX
!gdown https://drive.google.com/uc?id=1MIWx6wnrK4wMnZo-UD_7mdzE7KECIPnl
!gdown https://drive.google.com/uc?id=14MWXmmoKQfwcbFBUERXwN8-tvHqI85rL
!gdown https://drive.google.com/uc?id=1onMI7S8AbuQSXd6Rm4wbeT_t7PMdQJqe
!gdown https://drive.google.com/uc?id=1pwhNUP1v_1tEjpfL0hPZEYSvEjJSnFta
!gdown https://drive.google.com/uc?id=1fIZB6amYR4j2WclugemBs9Q0jPfpnYp-

Downloading...
From: https://drive.google.com/uc?id=1ggZc7kJMcHRiVH0EUAmQAOJYWgfLi1uX
To: /content/Google Pixel 9A Impressions.mp4
100% 20.0M/20.0M [00:00<00:00, 52.2MB/s]
Downloading...
From: https://drive.google.com/uc?id=1MIWx6wnrK4wMnZo-UD_7mdzE7KECIPnl
To: /content/[나의 아저씨 OST] Sondia - 어른 (Grown Ups) MV.mp4
100% 8.22M/8.22M [00:00<00:00, 97.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=14MWXmmoKQfwcbFBUERXwN8-tvHqI85rL
To: /content/Dream_Job.mp4
100% 6.66M/6.66M [00:00<00:00, 92.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=1onMI7S8AbuQSXd6Rm4wbeT_t7PMdQJqe
To: /content/Paul Kim(폴킴) _ Me After You(너를 만나).mp4
100% 8.56M/8.56M [00:00<00:00, 111MB/s]
Downloading...
From: https://drive.google.com/uc?id=1pwhNUP1v_1tEjpfL0hPZEYSvEjJSnFta
To: /content/Google AI Studio for Beginners.mp4
100% 5.63M/5.63M [00:00<00:00, 80.5MB/s]
Downloading...
From: https://drive.google.com/uc?id=1fIZB6amYR4j2WclugemBs9Q0jPfpnYp-
To: /content/Hugging Face + Langchain.mp4
100% 19.5

In [ ]:
dir_video_list = [f for f in os.listdir('/content') if f.endswith(".mp4")]
for video_file in dir_video_list:
    video_path = f'{VIDEO_DIR}/{video_file}'
    shutil.move(f'/content/{video_file}', video_path)
    filename = video_file.split('.mp4')[0]
    audio_path = os.path.join(AUDIO_DIR, f"{filename}.mp3")
    with VideoFileClip(video_path) as video_clip:
        video_clip.audio.write_audiofile(audio_path, codec="mp3", verbose=False, logger=None)

## 유튜브 요약 앱 고도화(자막 생성 및 번역)

- youtube_sub_tab() : "자막 생성"
    *   경로 구조화 -> 데이터 흐름 및 분리도 개선
        * VIDEO_DIR, SUB_DIR, AUDIO_DIR, CAPTURE_DIR, SUB_VIDEO_DIR
    *   영상 재사용 -> video list 활용



In [ ]:
# 유튜브 영상 다운로드 함수
def download_youtube_video(url, filename):
    save_path = os.path.join(VIDEO_DIR, f"{filename}.mp4")
    if os.path.exists(save_path):
        return f"⚠️ 파일 '{filename}' 이미 존재합니다."

    yt = YouTube(url)
    video_stream = yt.streams.filter(progressive=True, file_extension="mp4").order_by("resolution").desc().first()
    video_path = video_stream.download(output_path=VIDEO_DIR, filename=f"{filename}.mp4")

    # 영상에서 바로 오디오 추출
    audio_path = os.path.join(AUDIO_DIR, f"{filename}.mp3")
    with VideoFileClip(video_path) as video_clip:
        video_clip.audio.write_audiofile(audio_path, codec="mp3", verbose=False, logger=None)

    return f"✅ 영상 '{filename}' 업로드 완료."

In [ ]:
# 영상 목록 반환
def get_video_files():
    return [os.path.basename(f) for f in glob(os.path.join(VIDEO_DIR, "*.mp4"))]

def handle_download(url, name):
    name = name.split('.mp4')[0]
    message = download_youtube_video(url, name)
    updated_list = gr.update(choices=get_video_files(), value=f"{name}.mp4" if "✅" in message else None)
    show = gr.update(visible=True)
    return message, updated_list, show

In [ ]:
# 오디오 전사 함수
def transcribe_audio(audio_path):
    return transcriber.transcribe(audio_path)

In [ ]:
# 초 단위 시간(float)을 SRT 형식의 문자열로 변환
def format_timestamp(s):
    ms = int((s - int(s)) * 1000)
    h, m = int(s // 3600), int((s % 3600) // 60)
    sec = int(s % 60)
    return f"{h:02d}:{m:02d}:{sec:02d},{ms:03d}"

In [ ]:
# 원어 자막 파일 생성 함수
def create_subtitle_file(segments, path):
    try:
        with open(path, "w", encoding="utf-8") as f:
            for i, seg in enumerate(segments, 1):
                f.write(f"{i}\n{format_timestamp(seg['start'])} --> {format_timestamp(seg['end'])}\n{seg['text'].strip()}\n\n")
        return path
    except Exception as e:
        print(f"자막 파일 생성 실패: {e}")
        return f"자막 파일 생성 실패: {e}"

In [ ]:
def aistudio_call(client, model_name, prompt):
    for attempt in range(3):
        try:
            response = client.models.generate_content(model=model_name, contents=prompt)
            output = response.text.strip()
            break
        except Exception as e:
            time.sleep(random.uniform(3, 7))
    return output

In [ ]:
def translate_segments_and_create_srt(segs, src, tgt, key, summary, path, bs=5):
    c, m, n = genai.Client(api_key=key), "models/gemini-2.0-flash", 1
    with open(path, "w", encoding="utf-8-sig") as f:
        for i in tqdm(range(0, len(segs), bs), desc="자막 번역 중"):
            b = segs[i:i+bs]; p = '\n'.join([f'{j+1}. "{s["text"]}"' for j, s in enumerate(b)])
            o = aistudio_call(c, m, f'You are a translator.\nTranslate subtitles from {src} to {tgt} based on:\n"{summary}"\n{p}\nDo NOT add explanations, notes, or additional text.\n\n')
            r = re.findall(r"(\d+)\.\s*(.+)", o) or [(str(i+1), s["text"]) for i, s in enumerate(b)]
            for idx, txt in r:
                s = b[int(idx)-1]; st, et = format_timestamp(s['start']), format_timestamp(s['end'])
                f.write(f"{n}\n{st} --> {et}\n{txt.strip()}\n\n"); n += 1; time.sleep(random.uniform(3, 7))
    return path


In [ ]:
def create_subtitled_video_from_srt(v_path, s_path, o_path):
    v = VideoFileClip(v_path); w, h = v.size; clips = [v]; subs = pysrt.open(s_path, encoding="utf-8-sig")
    for s in subs:
        st, et = s.start.ordinal/1000, s.end.ordinal/1000; t = s.text.strip().replace("\n", " ")
        txt = TextClip(t, fontsize=15, font="NanumGothicBold", color="white", method="caption", size=(w*0.9,None), align="center")
        bg = ColorClip(size=(txt.w, txt.h), color=(0,0,0))
        sub_clip = CompositeVideoClip([bg, txt.set_position('center')]).set_position(("center", h - txt.h - 30)).set_opacity(0.7).set_start(st).set_end(et)
        clips.append(sub_clip)
    CompositeVideoClip(clips, size=v.size).write_videofile(o_path, codec="libx264", audio_codec="aac", verbose=False, logger=None, threads=4)
    return o_path


In [ ]:
def add_subtitles_ffmpeg(v_path, s_path, o_path):
    try:
        ffmpeg.input(v_path).output(
            o_path,
            vf=f"subtitles={s_path}:force_style='FontName=NanumGothicBold,FontSize=15'",
            **{'c:v': 'libx264', 'c:a': 'aac'}
        ).overwrite_output().run(capture_stdout=True, capture_stderr=True)
        print("영상 생성 완료:", o_path); return o_path
    except ffmpeg.Error as e:
        print("FFmpeg 오류:", e.stderr.decode('utf-8')); return None
    except Exception as e:
        print("기타 오류:", e); return None

In [ ]:
# 텍스트 요약 함수
def summarize_text(api_key, text, target_lang="ko", max_sents=5):
    try:
        client = genai.Client(api_key=api_key)
        prompt = (
            "The following is a transcription of a video.\n"
            f"Summarize the key points within {max_sents} sentences.\n"
            f"Write the summary in {target_lang}.\n\n"
            f"Transcription:\n{text}\n\n"
            "Summary:"
        )
        return client.models.generate_content(model="gemini-2.0-flash", contents=prompt).text.strip()
    except Exception as e:
        return f"요약 실패: {e}"

In [ ]:
# 통합 파이프라인 (번역 포함)
def process_pipeline(video_path, api_key, tgt_lang_name, summary_max_sents, check_case):
    video_name = video_path.split('.mp4')[0]
    video_path, audio_path = f'{VIDEO_DIR}/{video_path}', f"{AUDIO_DIR}/{video_name}.mp3"
    result = transcribe_audio(audio_path)
    src_lang = result.get('language', 'unknown')
    tgt_code = LANG_CODE.get(tgt_lang_name, 'ko')

    summary_target_lang = summarize_text(api_key, result['text'], target_lang=tgt_lang_name, max_sents=summary_max_sents)

    # 원본 자막용 경로
    orig_srt_path = os.path.join(SUB_DIR, f"{video_name}_subs_original_{src_lang}.srt")
    # 번역 자막용 경로
    trans_srt_path = os.path.join(SUB_DIR, f"{video_name}_subs_translated_{tgt_code}.srt")


    # 병렬 처리 함수 정의
    def create_original_subtitles():
        return create_subtitle_file(result['segments'], orig_srt_path)

    def create_translated_subtitles():
        # 요약 정보를 이용한 고품질 번역
        return translate_segments_and_create_srt(result['segments'], src_lang, tgt_code, api_key, summary_target_lang, trans_srt_path, 5)


    # 원어와 타겟 언어가 같다면 번역 불필요
    tasks = {"original": create_original_subtitles}
    if src_lang != tgt_code:
       tasks["translated"] = create_translated_subtitles

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(task_fn): task_name
            for task_name, task_fn in tasks.items()
        }
        results = {}
        for future in concurrent.futures.as_completed(futures):
            task_name = futures[future]
            results[task_name] = future.result()

    orig_srt_path = results.get("original", orig_srt_path)
    trans_srt_path = results.get("translated", None)

    # 영상 생성 작업 (멀티스레딩 추가)
    orig_vid_path = os.path.join(SUB_VIDEO_DIR, f"{video_name}_out_original_{src_lang}.mp4")
    trans_vid_path = os.path.join(SUB_VIDEO_DIR, f"{video_name}_out_translated_{tgt_code}.mp4")

    def create_video(video_path, srt_path, output_path):
        return create_subtitled_video_from_srt(video_path, srt_path, output_path)

    def fast_create_video(video_path, srt_path, output_path):
        return add_subtitles_ffmpeg(video_path, srt_path, output_path)

    video_fn = fast_create_video if check_case else create_video

    tasks_video = {"original_video": partial(video_fn, video_path, orig_srt_path, orig_vid_path)}
    if src_lang != tgt_code:
        tasks_video["translated_video"] = partial(video_fn, video_path, trans_srt_path, trans_vid_path)

    results = {}

    with concurrent.futures.ThreadPoolExecutor() as executor:
        futures = {
            executor.submit(task_fn): task_name
            for task_name, task_fn in tasks_video.items()
        }
        for future in concurrent.futures.as_completed(futures):
            task_name = futures[future]
            try:
                results[task_name] = future.result()
            except Exception as e:
                results[task_name] = None

    orig_vid_path = results.get("original_video", orig_vid_path)
    trans_vid_path = results.get("translated_video", None)

    return src_lang != tgt_code, summary_target_lang, orig_srt_path, trans_srt_path, orig_vid_path, trans_vid_path

In [ ]:
def process_pipeline_wrapper(video_list, key_input, lang_dropdown, summary_sents, fast_check):
    lang_check, summary, srt_path, trans_srt_path, video_path, trans_video_path = process_pipeline(video_list, key_input, lang_dropdown, summary_sents, fast_check)
    if trans_video_path is None:
        return (lang_check, summary, video_path, gr.update(visible=lang_check), srt_path, srt_path, video_path, video_path)
    else:
        return (lang_check, summary, trans_video_path, gr.update(visible=lang_check), srt_path, trans_srt_path, video_path, trans_video_path)

In [ ]:
def youtube_sub_tab():
    # with gr.Blocks() as block:
    with gr.Blocks(title="유튜브 영상 요약 및 자막 + 번역 서비스") as block:
        gr.Markdown("🎬 유튜브 요약 및 자막 서비스")

        with gr.Row():
            with gr.Column():
                url_input = gr.Textbox(label="유튜브 URL 입력")
                name_input = gr.Textbox(label="저장할 파일명 (예: myvideo.mp4)")
                download_output = gr.Textbox(label="업로드 상태", interactive=False, visible=False)
                download_btn = gr.Button("영상 업로드")

            with gr.Column():
                video_list = gr.Dropdown(choices=get_video_files(), label="영상 파일 선택")
                video_preview = gr.Video(label="영상 미리보기", autoplay=True)
                video_list.change(fn=lambda f: os.path.join(VIDEO_DIR, f), inputs=video_list, outputs=video_preview)
                with gr.Row():
                    key_input = gr.Textbox(label="Gemini API Key", type="password")
                    lang_dropdown = gr.Dropdown(label="Target Language", choices=list(LANG_CODE.keys()), value="한국어")
                    summary_sents = gr.Number(label="요약할 문장 갯수", value=5, precision=0)
                    fast_check = gr.Checkbox(label="빠른 처리 요청", value=False)
                run_btn = gr.Button("자막 생성")
        download_btn.click(fn=handle_download, inputs=[url_input, name_input], outputs=[download_output, video_list, download_output])
        gr.Markdown("---")
        with gr.Row():
            video_out = gr.Video(label="번역 자막 영상", autoplay=True)
            summary_out = gr.Textbox(label="요약", lines=10)

        with gr.Row(visible=True) as download_row:
            srt_download_btn = gr.DownloadButton(label="자막 파일 다운로드")
            video_download_btn = gr.DownloadButton(label="자막 영상 다운로드")

        with gr.Row(visible=False) as trans_download_row:
            trans_srt_download_btn = gr.DownloadButton(label="번역 자막 파일 다운로드")
            trans_video_download_btn = gr.DownloadButton(label="번역 자막 영상 다운로드")
        run_btn.click(fn=process_pipeline_wrapper, inputs=[video_list, key_input, lang_dropdown, summary_sents, fast_check],
                      outputs=[gr.State(), summary_out, video_out, trans_download_row, srt_download_btn, trans_srt_download_btn, video_download_btn, trans_video_download_btn])

    return block


## 정지 장면 캡쳐 기능 추가

- frame_capture_tab() : "프레임 캡처"
    *   timestamp 입력 기준 캡처 진행
    *   영문 결과 값 한국어로 변역 진행



In [ ]:
# 프레임 캡처 함수
def capture_frame(filename, timestamp, save_path):
    video_path = os.path.join(VIDEO_DIR, f"{filename}.mp4")
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_number = int(timestamp * fps)
    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
    ret, frame = cap.read()
    cap.release()
    if not ret:
        return None, "❌ 프레임 캡처 실패"
    cv2.imwrite(save_path, frame)
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    return frame_rgb, f"🖼️ {save_path}"

In [ ]:
def image_to_text(image_path):
    image = Image.open(image_path)
    messages = [
        {
            "role": "user",
            "content": [
                {"type": "image", "image": image_path},
                {"type": "text", "text": "Please describe the image in great detail as if you were explaining it to a visually impaired person. The image is a still shot from a video. Include all relevant visual information, such as the setting, colors, shapes, objects, and any actions or expressions that might be present."}]
            }
        ]
    text = ImageToTextProcessor.apply_chat_template(messages, add_generation_prompt=True, return_tensors="pt")
    inputs = ImageToTextProcessor(images=image, text=text, return_tensors="pt", padding=True, truncation=True).to(ImageToText.device)
    generated_ids = ImageToText.generate(**inputs, max_new_tokens=500)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
        ]
    generated_text = ImageToTextProcessor.batch_decode(generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False)
    return generated_text[0]

In [ ]:
# 한국어 번역 함수
def translation_text(api_key, text, target_lang="korean"):
    try:
        client = genai.Client(api_key=api_key)
        prompt = (
            "Below is a detailed description of an image, originally written in English."
            f"Please translate the description into {target_lang}, ensuring that the translation is accurate, fluent, and retains the full meaning of the original text."
            "Adapt the description appropriately for the nuances and cultural context of the target language."
            "Pay close attention to conveying any specific visual elements such as colors, shapes, and actions described in the original text. \n"
            f"IMAGE DESCRIPTION ::\n {text}"
        )
        return client.models.generate_content(model="gemini-2.0-flash", contents=prompt).text.strip()
    except Exception as e:
        return f"번역 실패: {e}"

In [ ]:
# 프레임 캡처 설명 함수
def capture_and_describe_frame(filename, hour, minute, second, api_key):
    filename = filename.split('.mp4')[0]
    timestamp = hour * 3600 + minute * 60 + second
    image_path = os.path.join(CAPTURE_DIR, f"{filename}_{hour}-{minute}-{second}.jpg")
    capture_image, result = capture_frame(filename, timestamp, image_path)
    if capture_image is None:
        return None, result
    else:
        try:
            generated_text = image_to_text(image_path)
            generated_text = translation_text(api_key, generated_text, target_lang="korean")
            return capture_image, generated_text
        except Exception as e:
            return capture_image, f"❌ 오류 발생: {str(e)}"

In [ ]:
# 유튜브 영상 다운로드 + 프레임 캡처 + 이미지 설명
def frame_capture_tab():
    with gr.Blocks(title="🎬 유튜브 영상 다운로드 및 프레임 캡처") as block:
        gr.Markdown("## 🎬 유튜브 영상 다운로드 및 프레임 캡처")

        with gr.Row():
            with gr.Column():
                url_input = gr.Textbox(label="유튜브 URL 입력")
                name_input = gr.Textbox(label="저장할 파일명 (예: myvideo.mp4)")
                download_output = gr.Textbox(label="업로드 상태", interactive=False, visible=False)
                download_btn = gr.Button("영상 업로드")

            with gr.Column():
                video_list = gr.Dropdown(choices=get_video_files(), label="영상 파일 선택")
                video_preview = gr.Video(label="영상 미리보기", autoplay=True)
                video_list.change(fn=lambda f: os.path.join(VIDEO_DIR, f), inputs=video_list, outputs=video_preview)
                with gr.Row():
                    hour = gr.Number(label="시", value=0, precision=0)
                    minute = gr.Number(label="분", value=0, precision=0)
                    second = gr.Number(label="초", value=0, precision=0)
                key_input = gr.Textbox(label="Gemini API Key", type="password")
                capture_btn = gr.Button("프레임 캡처")

        download_btn.click(fn=handle_download, inputs=[url_input, name_input], outputs=[download_output, video_list, download_output])

        gr.Markdown("---")
        gr.Markdown("### 🎞️ 캡처된 이미지와 설명")
        with gr.Row():
            output_image = gr.Image(label="캡처된 이미지")
            output_path = gr.Textbox(label="이미지 설명", interactive=False, lines=10)

        capture_btn.click(fn=capture_and_describe_frame, inputs=[video_list, hour, minute, second, key_input], outputs=[output_image, output_path])

    return block

## UI 구현

-  두 TAB으로 나누어서 구성
    *    "프레임 캡처"
    *    "자막 생성"



In [ ]:
# 앱 실행 구조 (탭 구성)
app = gr.TabbedInterface(
    [frame_capture_tab(), youtube_sub_tab()],
    tab_names=["프레임 캡처", "자막 생성"]
)
app.launch(debug=True)

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://851a7515710f7283f9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2181, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 1692, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/anyio/to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
           ^^^^^

영상 생성 완료: ./subtitles_videos/test2_out_translated_ko.mp4
영상 생성 완료: ./subtitles_videos/test2_out_original_en.mp4
